# Análise de mercado de veículos Honda Civic no estado da California

## Parte 2 - Data cleaning

## 1. Configuração inicial e importação de dados

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import os
import psycopg2
import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine, text

In [2]:
df = pd.read_csv("../data/raw/marketcheck.csv")

In [3]:
df.head()

,id,vin,heading,miles,data_source,vdp_url,carfax_1_owner,carfax_clean_title,exterior_color,interior_color,...,model_code,dealer.seller_email,mc_dealership.seller_email,availability_status,dealer.dealership_group_name,mc_dealership.mc_dealership_group_id,mc_dealership.mc_dealership_group_name,mc_dealership.mc_sub_dealership_group_id,mc_dealership.mc_sub_dealership_group_name,is_certified
0,2HGFG3B59CH544689-d6af690a-5502,2HGFG3B59CH544689,2012 Honda Civic Coupe,62853.0,mc,https://www.prcopowercars.com/listings/2012-ho...,False,False,Gray,Gray,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,19XFB2F87CE372279-ed867908-60fe,19XFB2F87CE372279,2012 Honda Civic Sedan,112727.0,mc,https://www.prcopowercars.com/listings/2012-ho...,False,False,No,No Color,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,19XFC2F54GE203507-90fbf3de-7d0d,19XFC2F54GE203507,2016 HONDA CIVIC SEDAN LX,127880.0,mc,https://www.rosevilleautomall.com/vehicle-deta...,False,True,NaN,NaN,...,FC2F5GEW,username@example.com,username@example.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2HGFC2F56HH502570-76980e60-7af0,2HGFC2F56HH502570,2017 Honda Civic Sedan LX,93077.0,mc,https://www.pacificautocenteroc.com/used-Costa...,False,False,Red,Black,...,FC2F5HEW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,19XFC2F71GE236045-8999218a-fef9,19XFC2F71GE236045,2016 Honda Civic Sedan,155108.0,mc,https://www.prcopowercars.com/listings/2016-ho...,False,False,No,No Color,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df.shape

(2414, 104)

## 2. Filtragem de colunas relevantes para análises

In [5]:
colunas = [
    'vin', 'price', 'msrp', 'ref_price', 'price_change_percent', 
    'miles', 'dom', 'dom_180', 'dom_active', 'dos_active', 
    'carfax_1_owner', 'dealer.id', 'dealer.dealer_type', 'dealer.city', 
    'exterior_color', 'interior_color', 'base_int_color', 'base_ext_color', 
    'build.year', 'build.trim', 'build.version', 'build.body_type', 'build.transmission', 
    'build.drivetrain', 'build.fuel_type', 'build.engine', 'build.engine_size', 'build.doors', 
    'build.cylinders', 'build.overall_height', 'build.overall_length', 'build.overall_width', 
    'build.highway_mpg', 'build.city_mpg', 'build.powertrain_type'
]

In [6]:
df = df[colunas]

In [7]:
df.columns

Index(['vin', 'price', 'msrp', 'ref_price', 'price_change_percent', 'miles',
       'dom', 'dom_180', 'dom_active', 'dos_active', 'carfax_1_owner',
       'dealer.id', 'dealer.dealer_type', 'dealer.city', 'exterior_color',
       'interior_color', 'base_int_color', 'base_ext_color', 'build.year',
       'build.trim', 'build.version', 'build.body_type', 'build.transmission',
       'build.drivetrain', 'build.fuel_type', 'build.engine',
       'build.engine_size', 'build.doors', 'build.cylinders',
       'build.overall_height', 'build.overall_length', 'build.overall_width',
       'build.highway_mpg', 'build.city_mpg', 'build.powertrain_type'],
      dtype='object')

## 3. Padronização técnica

### 3.1 Conversão para sistema métrico

In [8]:
pd.options.display.float_format = "{:.2f}".format

# Milhas para quilômetros
df['miles'] = (df['miles'] * 1.60934).round(0).astype('Int64')

# Milhas por galão (MPG) para quilômetros por litro (km/L)
df['build.highway_mpg'] = (df['build.highway_mpg'] * 0.4251).round(2)
df['build.city_mpg'] = (df['build.city_mpg'] * 0.4251).round(2)

# Polegadas para metros
df['build.overall_height'] = (df['build.overall_height'] * 0.0254).round(3)
df['build.overall_length'] = (df['build.overall_length'] * 0.0254).round(3)
df['build.overall_width']  = (df['build.overall_width'] * 0.0254).round(3)

### 3.2 Tradução das colunas para português

In [9]:
colunas_ptbr = {
    "vin": "chassi",
    "price": "preco_usd",
    "msrp": "preco_original_msrp_usd",
    "ref_price": "preco_referencia_usd",
    "price_change_percent": "percentual_mudanca_preco",
    "miles": "quilometragem",
    "dom": "dias_no_mercado_total",
    "dom_180": "dias_no_mercado_180d",
    "dom_active": "dias_no_mercado_ativo",
    "dos_active": "dias_desde_inicio_ativo",
    "carfax_1_owner": "unico_dono",
    "dealer.id": "id_revendedor",
    "dealer.dealer_type": "tipo_revendedor",
    "dealer.city": "cidade_revendedor",
    "mc_dealership.dealer_type": "tipo_revendedor_mc",
    "exterior_color": "cor_externa",
    "interior_color": "cor_interna",
    "base_int_color": "cor_interna_base",
    "base_ext_color": "cor_externa_base",
    "build.year": "ano_fabricacao",
    "build.trim": "versao",
    "build.version": "versao_detalhada",
    "build.body_type": "tipo_carroceria",
    "build.transmission": "transmissao",
    "build.drivetrain": "tracao",
    "build.fuel_type": "tipo_combustivel",
    "build.engine": "motor",
    "build.engine_size": "tamanho_motor",
    "build.doors": "portas",
    "build.cylinders": "cilindros",
    "build.overall_height": "altura_total",
    "build.overall_length": "comprimento_total",
    "build.overall_width": "largura_total",
    "build.highway_mpg": "consumo_estrada_mpg",
    "build.city_mpg": "consumo_cidade_mpg",
    "build.powertrain_type": "tipo_powertrain",
}

In [10]:
df.rename(columns=colunas_ptbr, inplace=True)

In [11]:
df.columns

Index(['chassi', 'preco_usd', 'preco_original_msrp_usd',
       'preco_referencia_usd', 'percentual_mudanca_preco', 'quilometragem',
       'dias_no_mercado_total', 'dias_no_mercado_180d',
       'dias_no_mercado_ativo', 'dias_desde_inicio_ativo', 'unico_dono',
       'id_revendedor', 'tipo_revendedor', 'cidade_revendedor', 'cor_externa',
       'cor_interna', 'cor_interna_base', 'cor_externa_base', 'ano_fabricacao',
       'versao', 'versao_detalhada', 'tipo_carroceria', 'transmissao',
       'tracao', 'tipo_combustivel', 'motor', 'tamanho_motor', 'portas',
       'cilindros', 'altura_total', 'comprimento_total', 'largura_total',
       'consumo_estrada_mpg', 'consumo_cidade_mpg', 'tipo_powertrain'],
      dtype='object')

### 3.3 Tradução dos valores da coluna 'combustíveis'

In [12]:
df['tipo_combustivel'] = df['tipo_combustivel'].replace({
    'Unleaded': 'Gasolina Comum',
    'Premium Unleaded': 'Gasolina Premium',
    'Electric / Unleaded': 'Eletrico / Gasolina',
    'Compressed Natural Gas': 'Gas Natural Comprimido'
})

### 3.4 Padronização de casas decimais das colunas com valores float

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2414 entries, 0 to 2413
Data columns (total 35 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   chassi                    2414 non-null   object 
 1   preco_usd                 2269 non-null   float64
 2   preco_original_msrp_usd   2269 non-null   float64
 3   preco_referencia_usd      1495 non-null   float64
 4   percentual_mudanca_preco  1476 non-null   float64
 5   quilometragem             2370 non-null   Int64  
 6   dias_no_mercado_total     2414 non-null   int64  
 7   dias_no_mercado_180d      2414 non-null   int64  
 8   dias_no_mercado_ativo     2414 non-null   int64  
 9   dias_desde_inicio_ativo   2414 non-null   int64  
 10  unico_dono                2220 non-null   object 
 11  id_revendedor             2414 non-null   int64  
 12  tipo_revendedor           2414 non-null   object 
 13  cidade_revendedor         2414 non-null   object 
 14  cor_exte

In [14]:
float_cols = df.select_dtypes(include='float64').columns
df[float_cols] = df[float_cols].round(2)

In [15]:
pd.set_option('display.max_columns', None)
df.head()

,chassi,preco_usd,preco_original_msrp_usd,preco_referencia_usd,percentual_mudanca_preco,quilometragem,dias_no_mercado_total,dias_no_mercado_180d,dias_no_mercado_ativo,dias_desde_inicio_ativo,unico_dono,id_revendedor,tipo_revendedor,cidade_revendedor,cor_externa,cor_interna,cor_interna_base,cor_externa_base,ano_fabricacao,versao,versao_detalhada,tipo_carroceria,transmissao,tracao,tipo_combustivel,motor,tamanho_motor,portas,cilindros,altura_total,comprimento_total,largura_total,consumo_estrada_mpg,consumo_cidade_mpg,tipo_powertrain
0,2HGFG3B59CH544689,NaN,NaN,NaN,NaN,101152,76,76,76,76,False,1026183,independent,Lafayette,Gray,Gray,Gray,Gray,2012,LX,LX Auto,Coupe,Automatic,FWD,Gasolina Comum,1.8L I4,1.80,2,4,1.40,4.52,1.80,17.43,12.75,Combustion
1,19XFB2F87CE372279,NaN,NaN,NaN,NaN,181416,79,27,27,27,False,1026183,independent,Lafayette,No,No Color,NaN,NaN,2012,EX,EX Auto,Sedan,Automatic,FWD,Gasolina Comum,1.8L I4,1.80,4,4,1.44,4.63,1.80,17.43,13.18,Combustion
2,19XFC2F54GE203507,16999.00,16999.00,16999.00,0.00,205802,93,20,20,19,False,1014272,franchise,Roseville,NaN,NaN,NaN,NaN,2016,LX,LX CVT,Sedan,CVT,FWD,Gasolina Comum,2.0L I4,2.00,4,4,1.44,4.63,1.80,17.43,13.18,Combustion
3,2HGFC2F56HH502570,12995.00,16995.00,16995.00,-23.54,149793,235,37,37,37,False,1103310,independent,Costa Mesa,Red,Black,Black,Red,2017,LX,LX CVT,Sedan,CVT,FWD,Gasolina Comum,2.0L I4,2.00,4,4,1.44,4.63,1.80,17.43,13.18,Combustion
4,19XFC2F71GE236045,NaN,NaN,NaN,NaN,249622,72,71,31,31,False,1026183,independent,Lafayette,No,No Color,NaN,NaN,2016,EX,EX CVT,Sedan,Automatic,FWD,Gasolina Comum,2.0L I4,2.00,4,4,1.44,4.63,1.80,17.43,13.18,Combustion


## 4. Tratamento de valores faltantes

### 4.1 Análise Inicial

In [16]:
valores_nulos = df.isnull().sum()
print(valores_nulos[valores_nulos > 0].sort_values(ascending=False))

percentual_mudanca_preco    938
preco_referencia_usd        919
cor_interna_base            571
cor_interna                 533
cor_externa_base            271
cor_externa                 225
unico_dono                  194
preco_original_msrp_usd     145
preco_usd                   145
quilometragem                44
versao_detalhada              8
dtype: int64


### 4.2 Quilometragem

Implementei uma estratégia de imputação hierárquica que prioriza a combinação cidade–ano ao estimar valores ausentes de quilometragem. Para garantir robustez, apliquei detecção de outliers com o critério 3×IQR antes do cálculo das medianas, de modo a preservar a variabilidade intrínseca dos dados e eliminar apenas observações estatisticamente discrepantes.

In [17]:
df['quilometragem'] = df['quilometragem'].astype('float64')

In [18]:
def imputacao_ano_cidade(df):
    df_imputado = df.copy()

    for (cidade, ano), grupo in df_imputado.groupby(['cidade_revendedor', 'ano_fabricacao']):
        valores_grupo = grupo['quilometragem'].dropna()
        nulos_grupo = grupo['quilometragem'].isna().sum()
        
        if nulos_grupo > 0:
            # Tentativa 1: Utiliza dados da própria cidade e ano
            if len(valores_grupo) >= 5:
                Q1 = valores_grupo.quantile(0.25)
                Q3 = valores_grupo.quantile(0.75)
                IQR = Q3 - Q1
                lower_bound = Q1 - 3.0 * IQR
                upper_bound = Q3 + 3.0 * IQR
                
                valores_filtrados = valores_grupo[
                    (valores_grupo >= lower_bound) & 
                    (valores_grupo <= upper_bound)
                ]
                
                mediana = valores_filtrados.median()
                
            else:
                # Tentativa 2: Se não houver dados suficientes na cidade, utiliza toda a região + ano
                dados_ano_estado = df_imputado[
                    (df_imputado['ano_fabricacao'] == ano) & 
                    (df_imputado['quilometragem'].notna())
                ]['quilometragem']
                
                if len(dados_ano_estado) > 10:
                    Q1_estado = dados_ano_estado.quantile(0.25)
                    Q3_estado = dados_ano_estado.quantile(0.75)
                    IQR_estado = Q3_estado - Q1_estado
                    
                    lower_bound_estado = Q1_estado - 3.0 * IQR_estado
                    upper_bound_estado = Q3_estado + 3.0 * IQR_estado
                    
                    valores_filtrados_estado = dados_ano_estado[
                        (dados_ano_estado >= lower_bound_estado) & 
                        (dados_ano_estado <= upper_bound_estado)
                    ]
                    
                    mediana = valores_filtrados_estado.median()

                else:
                    mediana = dados_ano_estado.median()
            
            mascara_nulos = (df_imputado['cidade_revendedor'] == cidade) & \
                           (df_imputado['ano_fabricacao'] == ano) & \
                           (df_imputado['quilometragem'].isna())
            
            df_imputado.loc[mascara_nulos, 'quilometragem'] = mediana
    
    return df_imputado

In [19]:
df = imputacao_ano_cidade(df)
nulos = df['quilometragem'].isna().sum()
print(f"Valores nulos restantes: {nulos}")

Valores nulos restantes: 0


### 4.3 Cores externas e internas

In [20]:
df['cor_externa'] = df['cor_externa'].fillna('Desconhecido')
df['cor_externa_base'] = df['cor_externa_base'].fillna(df['cor_externa'])

df['cor_interna'] = df['cor_interna'].fillna('Desconhecido')
df['cor_interna_base'] = df['cor_interna_base'].fillna(df['cor_interna'])

### 4.4 'versao_detalhada'

In [21]:
df['versao_detalhada'] = df['versao_detalhada'].fillna(df['versao'])

In [22]:
df['versao_detalhada'] = (
    df['versao_detalhada']
    .str.split('|')
    .str[0]
    .str.strip()
)
df['versao'] = df['versao'].replace({
    'HYBRID': 'Hybrid',
    'SI': 'Si'
})

### 4.5 'unico_dono'

In [23]:
df['unico_dono'] = df['unico_dono'].fillna('Desconhecido')
df['unico_dono'] = df['unico_dono'].replace({
    False: 'Não',
    True: 'Sim'
})

### 4.6 Sistema de preços

In [24]:
def imputar_precos_por_grupo(df):
    df_imputado = df.copy()
    carros_sem_preco = df_imputado[df_imputado['preco_usd'].isna()].index
    
    for idx in carros_sem_preco:
        carro = df_imputado.loc[idx]
        ano = carro['ano_fabricacao']
        versao = carro['versao']
        cidade = carro['cidade_revendedor']
        km = carro['quilometragem']
        
        preco_imputado = None
        
        # Estratégia 1: Mesmo ano + versão + cidade + faixa de KM (±20%)
        if not pd.isna(km):
            km_min = km * 0.8  # 80% da quilometragem original (20% menos)
            km_max = km * 1.2 # 120% da quilometragem original (20% mais)
            grupo1 = df_imputado[
                (df_imputado['ano_fabricacao'] == ano) &
                (df_imputado['versao'] == versao) &
                (df_imputado['cidade_revendedor'] == cidade) &
                (df_imputado['quilometragem'] >= km_min) &
                (df_imputado['quilometragem'] <= km_max) &
                (df_imputado['preco_usd'].notna())
            ]
            if len(grupo1) >= 3:
                preco_imputado = grupo1['preco_usd'].median()
        
        # Estratégia 2: Mesmo ano + versão + cidade
        if preco_imputado is None:
            grupo2 = df_imputado[
                (df_imputado['ano_fabricacao'] == ano) &
                (df_imputado['versao'] == versao) &
                (df_imputado['cidade_revendedor'] == cidade) &
                (df_imputado['preco_usd'].notna())
            ]
            if len(grupo2) >= 3:
                preco_imputado = grupo2['preco_usd'].median()
        
        # Estratégia 3: Mesmo ano + versão
        if preco_imputado is None:
            grupo3 = df_imputado[
                (df_imputado['ano_fabricacao'] == ano) &
                (df_imputado['versao'] == versao) &
                (df_imputado['preco_usd'].notna())
            ]
            if len(grupo3) >= 3:
                preco_imputado = grupo3['preco_usd'].median()
        
        # Estratégia 4: Mesmo ano
        if preco_imputado is None:
            grupo4 = df_imputado[
                (df_imputado['ano_fabricacao'] == ano) &
                (df_imputado['preco_usd'].notna())
            ]
            if len(grupo4) >= 3:
                preco_imputado = grupo4['preco_usd'].median()
        
        # Estratégia 5: Mediana geral (último recurso)
        if preco_imputado is None:
            preco_imputado = df_imputado['preco_usd'].median()
        
        df_imputado.loc[idx, 'preco_usd'] = preco_imputado
        df_imputado.loc[idx, 'preco_original_msrp_usd'] = preco_imputado
    
    return df_imputado

df = imputar_precos_por_grupo(df)

### 4.6.1 preco_usd

In [25]:
df['preco_usd'] = df['preco_usd'].round(2)
print(f"preco_usd nulos: {df['preco_usd'].isna().sum()}")

preco_usd nulos: 0


### 4.6.2 preco_original_msrp_usd

In [26]:
df['preco_original_msrp_usd'] = df['preco_original_msrp_usd'].round(2)
print(f"preco_original_msrp_usd nulos: {df['preco_original_msrp_usd'].isna().sum()}")

preco_original_msrp_usd nulos: 0


### 4.6.3 preco_referencia_usd

In [27]:
df['preco_referencia_usd'] = df['preco_referencia_usd'].fillna(df['preco_usd'])
print(f"preco_referencia_usd nulos: {df['preco_referencia_usd'].isna().sum()}")

preco_referencia_usd nulos: 0


### 4.6.4 percentual_mudanca_preco

In [28]:
df['percentual_mudanca_preco'] = (
    (df['preco_usd'] - df['preco_referencia_usd']) / 
    df['preco_referencia_usd'] * 100
).round(2)

print(f"percentual_mudanca_preco nulos: {df['percentual_mudanca_preco'].isna().sum()}")

percentual_mudanca_preco nulos: 0


## 5. Categorização e enriquecimento

### 5.1 Categorização de 'cor_externa'

In [29]:
# Limpeza e padronização de cores externas
df['cor_externa'] = df['cor_externa'].str.title()
df['cor_externa'] = df['cor_externa'].replace([
    'No', 'Unknown', 'Not Available', 'N / A', 'Other'
], 'Desconhecido')

# Padronização de variações
df['cor_externa'] = df['cor_externa'].replace({
    'Grey': 'Gray', 'Dk. Gray': 'Gray', 'Dk. Grey': 'Gray', 
    'Silver Gray': 'Silver', 'Silver/Gray': 'Silver'
})

# Limpeza de caracteres especiais
df['cor_externa'] = df['cor_externa'].str.replace(r'\s*\-.*', '', regex=True)
df['cor_externa'] = df['cor_externa'].str.replace(r'\s*\(.*', '', regex=True)
df['cor_externa'] = df['cor_externa'].str.replace(r'\s*\/.*', '', regex=True)
df['cor_externa'] = df['cor_externa'].str.replace(r'\s*\d+.*', '', regex=True)
df['cor_externa'] = df['cor_externa'].str.replace('"', '').str.replace("'", '')

# Categorização em cores base
def categorizar_cor(cor):
    cores_base = {
        'Branco': ['White', 'Pearl White', 'Taffeta White', 'Platinum White', 'White Orchid Pearl'],
        'Preto': ['Black', 'Crystal Black', 'Crystal Black Pearl'],
        'Cinza': ['Gray', 'Modern Steel', 'Meteorite Gray', 'Urban Titanium', 'Polished Metal Metallic', 'Charcoal'],
        'Prata': ['Silver', 'Lunar Silver', 'Alabaster Silver', 'Cool Mist Metallic'],
        'Azul': ['Blue', 'Aegean Blue', 'Cosmic Blue', 'Dyno Blue'],
        'Vermelho': ['Red', 'Rallye Red', 'Crimson Pearl', 'Burgundy', 'Burgundy Night Pearl', 'Molten Lava Pearl'],
        'Marrom': ['Brown', 'Kona Coffee Metallic'],
        'Outras': ['Green', 'Orange', 'Yellow', 'Purple', 'Gold', 'Bronze', 'Teal', 'Pewter', 'Platinum', 
                  'Frozen Grey', 'Maroon', 'Beige', 'Pear', 'Burgandy']
    }
    
    for base, variantes in cores_base.items():
        if any(variante in cor for variante in variantes):
            return base
    return 'Outras'

df['cor_externa_base'] = df['cor_externa'].apply(categorizar_cor)

print(df['cor_externa_base'].value_counts())

cor_externa_base
Branco      523
Cinza       514
Preto       368
Outras      298
Prata       282
Azul        252
Vermelho    162
Marrom       15
Name: count, dtype: int64


### 5.2 Categorização de 'cor_interna'

In [30]:
df['cor_interna'] = df['cor_interna'].str.title()

df['cor_interna'] = df['cor_interna'].replace({
    'No Color': 'Desconhecido', 'Unknown': 'Desconhecido', 'Other': 'Desconhecido',
    'Gone': 'Desconhecido', 'NONJACKASSSTAFF': 'Desconhecido', 'NONJACKASS': 'Desconhecido',
    'Lx': 'Desconhecido', 'Ex-L': 'Desconhecido'
})

# Remoção de materiais e termos extras
df['cor_interna'] = df['cor_interna'].str.replace(r'\s*Cloth\s*', '', regex=True)
df['cor_interna'] = df['cor_interna'].str.replace(r'\s*Leather\s*', '', regex=True)
df['cor_interna'] = df['cor_interna'].str.replace(r'\s*Trim\s*', '', regex=True)
df['cor_interna'] = df['cor_interna'].str.replace(r'\s*Suede\s*', '', regex=True)
df['cor_interna'] = df['cor_interna'].str.replace(r'\s*With\s*', '', regex=True)
df['cor_interna'] = df['cor_interna'].str.replace(r'\s*with\s*', '', regex=True)
df['cor_interna'] = df['cor_interna'].str.replace(r'^\-', '', regex=True)
df['cor_interna'] = df['cor_interna'].str.replace(r'\/.*', '', regex=True)
df['cor_interna'] = df['cor_interna'].str.strip()

df['cor_interna'] = df['cor_interna'].apply(lambda x: 'Desconhecido' if x == '' else x)

def categorizar_cor_interna(cor):
    cores_base = {
        'Black': ['Black', 'Blk', 'Dark', 'Charcoal', 'Nonjackassstaff', 'Nonjackass', 'Blk Clth'],
        'Gray': ['Gray', 'Grey', 'Stone'],
        'Beige': ['Beige', 'Tan', 'Ivory', 'Cream', 'Champagne', 'Off-White'],
        'Brown': ['Brown'],
        'White': ['White'],
        'Red': ['Red'],
        'Silver': ['Silver'],
        'Desconhecido': ['Desconhecido']
    }
    
    for base, variantes in cores_base.items():
        if any(variante.lower() in cor.lower() for variante in variantes):
            return base
    return 'Desconhecido'

df['cor_interna_base'] = df['cor_interna'].apply(categorizar_cor_interna)

print(df['cor_interna_base'].unique())

['Gray' 'Desconhecido' 'Black' 'Silver' 'Beige' 'Brown' 'Red' 'White']


## 5.4 Correções

### 5.4.1 Correção técnica dos valores da coluna 'tração'

In [31]:
# Honda Civic é exclusivamente FWD
df.loc[df['tracao'] == '4WD', 'tracao'] = 'FWD'
df.loc[df['tracao'] == 'RWD', 'tracao'] = 'FWD'

df['tracao'].unique()

array(['FWD'], dtype=object)

## 6. Carregando os dados em uma database PostgreSQL

In [32]:
DB_HOST = "localhost"
DB_PORT = "5432"
DB_USER = "postgres"
DB_PASSWORD = "senha123"
DB_NAME = "civic_db"
MASTER_DB = "postgres"

# 1. Cria a database caso não exista
try:
    conn = psycopg2.connect(
        host=DB_HOST,
        port=DB_PORT,
        database=MASTER_DB,
        user=DB_USER,
        password=DB_PASSWORD
    )
    conn.set_session(autocommit=True)
    cur = conn.cursor()

    cur.execute("SELECT 1 FROM pg_database WHERE datname = %s;", (DB_NAME,))
    exists = cur.fetchone()

    if not exists:
        cur.execute(f"CREATE DATABASE {DB_NAME};")
        print(f"Database '{DB_NAME}' criada!")
    else:
        print(f"Database '{DB_NAME}' já existe!")

    cur.close()
    conn.close()

except Exception as e:
    print(f"Erro ao criar/verificar database: {e}")
    exit()

# 2. Conecta ao novo banco de dados
engine = create_engine(f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

# 3. Cria schema relacional no PostgreSQL
with engine.begin() as conn:
    conn.execute(text("""
        DROP TABLE IF EXISTS veiculos CASCADE;
        DROP TABLE IF EXISTS revendedores CASCADE;

        CREATE TABLE revendedores (
            id_revendedor      BIGINT PRIMARY KEY,
            tipo_revendedor    VARCHAR(50),
            cidade_revendedor  VARCHAR(50)
        );

        CREATE TABLE veiculos (
            chassi                      VARCHAR(20) PRIMARY KEY,
            id_revendedor               BIGINT REFERENCES revendedores(id_revendedor),

            ano_fabricacao              SMALLINT,
            versao                      VARCHAR(20),
            versao_detalhada            TEXT,

            preco_usd                   NUMERIC(10,2),
            preco_original_msrp_usd     NUMERIC(10,2),
            preco_referencia_usd        NUMERIC(10,2),
            percentual_mudanca_preco    NUMERIC(6,3),

            quilometragem               INTEGER,
            dias_no_mercado_total       INTEGER,
            dias_no_mercado_180d        INTEGER,
            dias_no_mercado_ativo       INTEGER,
            dias_desde_inicio_ativo     INTEGER,

            unico_dono                  VARCHAR(15),

            tipo_carroceria             VARCHAR(10),
            transmissao                 VARCHAR(15),
            tracao                      VARCHAR(5),
            tipo_combustivel            VARCHAR(30),
            motor                       VARCHAR(20),
            tamanho_motor               NUMERIC(3,1),
            portas                      SMALLINT,
            cilindros                   SMALLINT,
            altura_total                NUMERIC(3,2),
            comprimento_total           NUMERIC(3,2),
            largura_total               NUMERIC(3,2),
            consumo_estrada_mpg         NUMERIC(4,2),
            consumo_cidade_mpg          NUMERIC(4,2),
            tipo_powertrain             VARCHAR(15),
            cor_externa                 VARCHAR(30),
            cor_interna                 VARCHAR(30),
            cor_externa_base            VARCHAR(30),
            cor_interna_base            VARCHAR(30)
        );
    """))

print("Estrutura relacional criada no banco!")

# 4. Prepara dataframes
# Revendedores (tabela de dimensão)
df_revendedores = df[['id_revendedor', 'tipo_revendedor', 'cidade_revendedor']].drop_duplicates('id_revendedor')

# Veículos (tabela principal)
veiculos_cols = [
    'chassi', 'id_revendedor', 'ano_fabricacao', 'versao', 'versao_detalhada',
    'tipo_carroceria', 'transmissao', 'tracao', 'tipo_combustivel', 'motor',
    'tamanho_motor', 'cilindros', 'portas', 'tipo_powertrain',
    'preco_usd', 'preco_original_msrp_usd', 'preco_referencia_usd', 'percentual_mudanca_preco',
    'cor_externa', 'cor_interna', 'cor_interna_base', 'cor_externa_base',
    'quilometragem', 'dias_no_mercado_total', 'dias_no_mercado_180d',
    'dias_no_mercado_ativo', 'dias_desde_inicio_ativo', 'unico_dono',
    'altura_total', 'comprimento_total', 'largura_total',
    'consumo_estrada_mpg', 'consumo_cidade_mpg'
]
df_veiculos = df[veiculos_cols].copy()

# 5. Exporta dados para o banco
with engine.begin() as conn:
    df_revendedores.to_sql('revendedores', conn, if_exists='append', index=False)
    df_veiculos.to_sql('veiculos', conn, if_exists='append', index=False)

print("Dados exportados!")

Database 'civic_db' criada!
Estrutura relacional criada no banco!
Dados exportados!


## 7. Exportação do dataset para tabela CSV

In [33]:
df.to_csv("../data/processed/veiculos_full_data.csv", index=False)

## Créditos e contato

**Desenvolvido por:**  
Bruno Casini

**GitHub:**  
[<img src="https://img.icons8.com/ios-filled/20/000000/github.png"/> GitHub](https://github.com/kzini)  
`https://github.com/kzini`

**LinkedIn:**  
[<img src="https://img.icons8.com/ios-filled/20/000000/linkedin.png"/> LinkedIn](https://www.linkedin.com/in/kzini)  
`https://www.linkedin.com/in/kzini`